
# Football Analysis Playground

Welcome! This notebook provides an interactive way to explore football data using the `api-football` service. It includes a suite of applications for live scores, team details, match analysis, and more.

**How to use:** Run the cells sequentially by pressing `Shift + Enter`. The final cell will start an interactive menu.


In [1]:

# Install necessary libraries
import sys
!{sys.executable} -m pip install requests pandas



[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:

# All necessary imports
import os
import requests
import pandas as pd
from IPython.display import display
from dotenv import load_dotenv
load_dotenv()


True

In [3]:

# API Client Class
class FootballAPIClient:
    """
    A client for interacting with the API-Football API.
    """
    def __init__(self, api_key):
        self.api_key = api_key
        self.headers = {
            'x-rapidapi-key': self.api_key,
            'x-rapidapi-host': 'v3.football.api-sports.io'
        }
        self.base_url = 'https://v3.football.api-sports.io'

    def _make_request(self, endpoint, params=None):
        url = f"{self.base_url}{endpoint}"
        try:
            response = requests.get(url, headers=self.headers, params=params)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"An error occurred: {e}")
            return None

    def get_live_fixtures(self):
        return self._make_request('/fixtures', params={'live': 'all'})

    def get_team_info(self, team_id):
        return self._make_request('/teams', params={'id': team_id})

    def get_players(self, team_id, season=2023):
        return self._make_request('/players', params={'team': team_id, 'season': season})

    def get_fixture_events(self, fixture_id):
        return self._make_request('/fixtures/events', params={'fixture': fixture_id})

    def get_fixture_statistics(self, fixture_id):
        return self._make_request('/fixtures/statistics', params={'fixture': fixture_id})

    def get_standings(self, league_id, season):
        return self._make_request('/standings', params={'league': league_id, 'season': season})

    def get_top_scorers(self, league_id, season):
        return self._make_request('/topscorers', params={'league': league_id, 'season': season})

    def get_predictions(self, fixture_id):
        return self._make_request('/predictions', params={'fixture': fixture_id})

    def get_odds(self, fixture_id):
        return self._make_request('/odds', params={'fixture': fixture_id})


In [4]:

# Football Applications Class
class FootballApps:
    def __init__(self, api_client):
        self.api_client = api_client

    def live_score_app(self):
        print("--- Live Score App ---")
        fixtures = self.api_client.get_live_fixtures()
        if fixtures and fixtures.get('response'):
            if not fixtures['response']:
                print("No live games at the moment.")
                return
            for fixture in fixtures['response']:
                teams = fixture['teams']
                goals = fixture['goals']
                print(f"{teams['home']['name']} {goals['home']} - {goals['away']} {teams['away']['name']}")
        else:
            print("Could not retrieve live scores.")

    def team_page_app(self, team_id):
        print(f"--- Team Page for Team ID: {team_id} ---")
        team_info = self.api_client.get_team_info(team_id)
        if team_info and team_info.get('response'):
            team = team_info['response'][0]['team']
            venue = team_info['response'][0]['venue']
            print(f"Team: {team['name']}, Founded: {team['founded']}")
            print(f"Venue: {venue['name']}, Capacity: {venue['capacity']}")
        else:
            print("Could not retrieve team information.")

        print("\n--- Players ---")
        players_info = self.api_client.get_players(team_id)
        if players_info and players_info.get('response'):
            players_df = pd.DataFrame([p['player'] for p in players_info['response']])
            display(players_df[['name', 'age', 'nationality', 'height', 'weight']])
        else:
            print("Could not retrieve player information.")

    def match_center_app(self, fixture_id):
        print(f"--- Match Center for Fixture ID: {fixture_id} ---")
        events = self.api_client.get_fixture_events(fixture_id)
        if events and events.get('response'):
            print("\n--- Events ---")
            for event in events['response']:
                print(f"{event['time']['elapsed']}' {event['type']} - {event['detail']} ({event['team']['name']})")
        else:
            print("Could not retrieve match events.")

        print("\n--- Statistics ---")
        stats = self.api_client.get_fixture_statistics(fixture_id)
        if stats and stats.get('response') and len(stats.get('response')) >= 2:
            home_stats = {s['type']: s['value'] for s in stats['response'][0]['statistics']}
            away_stats = {s['type']: s['value'] for s in stats['response'][1]['statistics']}
            stats_df = pd.DataFrame([home_stats, away_stats], index=[stats['response'][0]['team']['name'], stats['response'][1]['team']['name']])
            display(stats_df)
        else:
            print("Could not retrieve match statistics.")

    def league_dashboard_app(self, league_id, season):
        print(f"--- League Dashboard for League ID: {league_id}, Season: {season} ---")
        standings = self.api_client.get_standings(league_id, season)
        if standings and standings.get('response'):
            print("\n--- Standings ---")
            standings_list = [{'Rank': s['rank'], 'Team': s['team']['name'], 'Points': s['points'], 'GD': s['goalsDiff']} for s in standings['response'][0]['league']['standings'][0]]
            standings_df = pd.DataFrame(standings_list)
            display(standings_df)
        else:
            print("Could not retrieve standings.")

        print("\n--- Top Scorers ---")
        top_scorers = self.api_client.get_top_scorers(league_id, season)
        if top_scorers and top_scorers.get('response'):
            scorers_list = [{'Player': s['player']['name'], 'Goals': s['statistics'][0]['goals']['total']} for s in top_scorers['response']]
            scorers_df = pd.DataFrame(scorers_list)
            display(scorers_df)
        else:
            print("Could not retrieve top scorers.")

    def betting_helper_app(self, fixture_id):
        print(f"--- Betting Helper for Fixture ID: {fixture_id} ---")
        predictions = self.api_client.get_predictions(fixture_id)
        if predictions and predictions.get('response'):
            print("\n--- Predictions ---")
            prediction = predictions['response'][0]
            print(f"Winner: {prediction['predictions']['winner']['name']}")
            print(f"Chance of Home Win: {prediction['predictions']['percent']['home']}")
            print(f"Chance of Draw: {prediction['predictions']['percent']['draw']}")
            print(f"Chance of Away Win: {prediction['predictions']['percent']['away']}")
        else:
            print("Could not retrieve predictions.")

        print("\n--- Odds ---")
        odds = self.api_client.get_odds(fixture_id)
        if odds and odds.get('response'):
            if not odds['response']:
                print("No odds available for this fixture.")
                return
            bookmaker = odds['response'][0]['bookmakers'][0]
            print(f"Bookmaker: {bookmaker['name']}")
            for bet in bookmaker['bets']:
                if bet['name'] == 'Match Winner':
                    home = bet['values'][0]
                    draw = bet['values'][1]
                    away = bet['values'][2]
                    print(f"Match Winner Odds: {home['value']}({home['odd']}), {draw['value']}({draw['odd']}), {away['value']}({away['odd']})")
        else:
            print("Could not retrieve odds.")


In [17]:

# Main Function to run the apps
def run_apps():
    # IMPORTANT: Replace with your own API key.
    # For your security, it's best to load this from a configuration file or environment variable.
    API_KEY = os.getenv("FOOTBALL_API_USERNAME")

    if not API_KEY or API_KEY == "YOUR_API_KEY":
        print("Please replace 'YOUR_API_KEY' with your actual API key from api-football.com")
        return

    client = FootballAPIClient(api_key=API_KEY)
    apps = FootballApps(api_client=client)

    print("--- Football Apps Menu ---")
    print("1. Live Score App")
    print("2. Team Page App")
    print("3. Match Center App")
    print("4. League Dashboard App")
    print("5. Betting Helper App")
    print("0. Exit")

    while True:
        choice = input("\nChoose an app to run (or 0 to exit): ")
        if choice == '1':
            apps.live_score_app()
        elif choice == '2':
            team_id = int(input("Enter Team ID (e.g., 33 for Man United): "))
            apps.team_page_app(team_id)
        elif choice == '3':
            fixture_id = int(input("Enter Fixture ID: "))
            apps.match_center_app(fixture_id)
        elif choice == '4':
            league_id = int(input("Enter League ID (e.g., 39 for Premier League): "))
            season = int(input("Enter Season (e.g., 2023): "))
            apps.league_dashboard_app(league_id, season)
        elif choice == '5':
            fixture_id = int(input("Enter Fixture ID: "))
            apps.betting_helper_app(fixture_id)
        elif choice == '0':
            print("Exiting.")
            break
        else:
            print("Invalid choice. Please try again.")



In [18]:

# Run the main application
run_apps()


--- Football Apps Menu ---
1. Live Score App
2. Team Page App
3. Match Center App
4. League Dashboard App
5. Betting Helper App
0. Exit
Exiting.
